In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib as mpl

import cv2
import os
import pickle as pic
from collections import deque
import sys

loss_function = tf.keras.losses.Huber()
initializer = tf.keras.initializers.VarianceScaling(scale=2.0)

tf.__version__

In [ ]:
learning_rate=0.00025

loss_function = tf.keras.losses.Huber()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0)

In [ ]:
def create_network(learning_rate, action_space):	
    initializer = tf.keras.initializers.VarianceScaling(scale=2.0)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input(shape=(105,80,4)))
    model.add(tf.keras.layers.Conv2D(32, 8, padding="same", strides=4, activation="relu", kernel_initializer=initializer, use_bias=False, name = "conv2d_1"))
    model.add(tf.keras.layers.Conv2D(64, 4, padding="same", strides=2, activation="relu", kernel_initializer=initializer, use_bias=False, name = "conv2d_2"))
    model.add(tf.keras.layers.Conv2D(64, 3, padding="same", strides=1, activation="relu", kernel_initializer=initializer, use_bias=False, name = "conv2d_3"))
    model.add(tf.keras.layers.Flatten(name = "flatten_1"))
    model.add(tf.keras.layers.Dense(512,activation="relu", kernel_initializer=initializer, name = "dense_1"))
    model.add(tf.keras.layers.Dense(action_space, kernel_initializer=initializer, name = "dense_2"))

    return model

In [ ]:
Y!wget http://www.atarimania.com/roms/Roms.rar
!mkdir ROM
!unrar e Roms.rar ROM
!python -m atari_py.import_roms ROM

In [ ]:
import gym

In [ ]:
env = gym.make('Pong-v0')

model = create_network(learning_rate, env.action_space.n)
model.compile(loss=loss_function, optimizer=optimizer)

target_model = create_network(learning_rate, env.action_space.n)
target_model.compile(loss=loss_function, optimizer=optimizer)

In [ ]:
model.summary()

In [ ]:
target_model.summary()

In [ ]:
env = gym.make('Pong-v0')
env.reset()

action = env.action_space.sample() 
state, reward, done, info = env.step(action)

print(state.shape)
print(state.min(), state.max())

In [ ]:
print(state)

In [ ]:
mpl.rcParams['figure.dpi'] = 300

plt.imshow(state)
plt.savefig('pong.png')

In [ ]:
def screen_pixel_preprocess(observation):
    s = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
    s = cv2.resize(s, (0, 0), fx=0.5, fy=0.5, interpolation = cv2.INTER_AREA) 
    s = s/236.0
    return s

In [ ]:
state = screen_pixel_preprocess(state)
state.shape

In [ ]:
print(state)

In [ ]:
plt.imshow(state, cmap='gray')
plt.savefig('pong2.png')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.chdir("drive/My Drive/colabpro_drive")

In [ ]:
pwd

# Render OpenAI Gym on Google Colab

In [ ]:
!apt update
!apt install xvfb

In [ ]:
nb_path = "/content/drive/My Drive/colabpro_drive/lib"
sys.path.append(nb_path)

In [ ]:
!pip install pyvirtualdisplay --target="{nb_path}" --upgrade 

In [ ]:
import pyvirtualdisplay
import gym
from gym.wrappers import Monitor
import glob
import io
import base64
from IPython import display as ipythondisplay
from IPython.display import HTML

In [ ]:
d = pyvirtualdisplay.Display()
d.start()

In [ ]:
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay loop controls style="height: 400px;">
      <source src="data:video/mp4;base64,{0}" type="video/mp4" />
      </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [ ]:
env = wrap_env(gym.make('Pong-v0'))
state = env.reset()

while True:
    env.render()
    action = env.action_space.sample() 
    state, reward, done, info = env.step(action) 
        
    if done: 
        break;
            
env.close()
show_video()

#Exploration Strategy

In [ ]:
epsilon = 1.0
epsilon_min = 0.1
epsilon_max = 1.0
epsilon_interval = (
    epsilon_max - epsilon_min
)

epsilon_greedy_frames = 1000000.0
epsilon_random_frames = 50000

eps_memory = []

In [ ]:
for frame_count in range(2000000):
  if frame_count < epsilon_random_frames:
    eps_memory.append(1)
  else:  
    eps_memory.append(epsilon)

  epsilon -= epsilon_interval/epsilon_greedy_frames
  epsilon = max(epsilon, epsilon_min)

In [ ]:
import plotly.graph_objs as go
import pickle as p
import numpy as np

In [ ]:
h1 = go.Scatter(y=eps_memory, 
                    mode="lines", line=dict(
                    width=2,
                    color='blue'),
                    name="epsilon"
                   )

data = [h1]
layout1 = go.Layout(title='Epsilon Schedule',
                   xaxis=dict(title='Frame Count'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data = data, layout=layout1)
fig1.show(renderer="colab")

# Collect Dataset

In [ ]:
frame_id = 0
state = []
pre_state = []

def sample():
  return np.random.choice(6)

def step(action):
  global frame_id
  frame_id+=1
  return frame_id

def update_state(state, observation):
    state.append(observation)

    if len(state) > 4:
        del state[:1]

def predict(state):
  return 1

In [ ]:
update_state(state, None)

for num_step in range(5):
  if len(state) < 4:
      action = sample()
  else:
      action = predict(state)

  pre_state.append(state[-1])
  if len(pre_state) > 4:
      del pre_state[:1]
  
  observation = step(action)

  update_state(state,observation)

In [ ]:
pre_state, state

# **Double DQN**

In [ ]:
def update_state(state, observation):
    observation = screen_pixel_preprocess(observation)
    state.append(observation)

    if len(state) > 4:
        del state[:1]

In [ ]:
def save_history(filename, history):
    with open(filename, 'wb') as file:
      pic.dump(history, file)

In [ ]:
action_memory = []
state_memory = []
state_next_memory = []
rewards_memory = []
done_memory = []
history= []

num_episode = 2000
frame_count = 0

epsilon_random_frames = 50000
epsilon_greedy_frames = 1000000.0
max_memory_length = 40000
update_after_actions = 4
update_target_network = 10000

num_action = env.action_space.n
batch_size = 32

gamma = 0.99  # Discount factor for past rewards
epsilon = 1.0
epsilon_min = 0.1
epsilon_max = 1.0
epsilon_interval = (
    epsilon_max - epsilon_min
)

In [ ]:
for i in range(num_episode):
    observation = env.reset()
    done = False
    state = []
    pre_state = []
    update_state(state,observation)

    episode_reward = 0

    while not done: 
        frame_count += 1
        if frame_count < epsilon_random_frames or epsilon > random.random() or len(state) < 4:  
            action = np.random.choice(num_action)
        else:
            s = np.stack((state[0],state[1],state[2],state[3]),axis=2)
            s = np.array([s])

            action_probs = model.predict(s)
            action = tf.argmax(action_probs[0]).numpy()

        epsilon -= epsilon_interval/epsilon_greedy_frames
        epsilon = max(epsilon, epsilon_min)

        pre_state.append(state[-1])
        if len(pre_state) > 4:
            del pre_state[:1]

        observation, reward, done, _ = env.step(action)

        update_state(state, observation)


        episode_reward += reward

        if len(state) == 4 and len(pre_state) == 4:
            action_memory.append(action)

            pre_state_for_model = np.stack((pre_state[0],pre_state[1],pre_state[2],pre_state[3]),axis=2)
            state_memory.append(pre_state_for_model)

            state_for_model = np.stack((state[0],state[1],state[2],state[3]),axis=2)
            state_next_memory.append(state_for_model)

            rewards_memory.append(reward)
        
        if frame_count % update_after_actions == 0 and len(rewards_memory) > batch_size:
            random_index = np.random.choice(range(len(rewards_memory)), size=batch_size)

            action_sample = [action_memory[i] for i in random_index]
            state_sample = np.array([state_memory[i] for i in random_index])
            state_next_sample = np.array([state_next_memory[i] for i in random_index])
            rewards_sample = [rewards_memory[i] for i in random_index]
            
            future_rewards = target_model.predict(state_next_sample)
            next_q_values = rewards_sample + gamma * tf.reduce_max(
                future_rewards, axis=1
            )

            masks = tf.one_hot(action_sample, num_action)
            
            with tf.GradientTape() as tape:
                q_values = model(state_sample)
                q_action = tf.reduce_sum(tf.multiply(q_values, masks), axis=1)

                loss = loss_function(next_q_values, q_action)
                grads = tape.gradient(loss, model.trainable_variables)
                optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if frame_count % update_target_network == 0:
            target_model.set_weights(model.get_weights())

        if len(rewards_memory) > max_memory_length:
            del action_memory[:1]
            del state_memory[:1]
            del state_next_memory[:1]
            del rewards_memory[:1]    

    history.append(episode_reward)        
    print(str(i)+ " episode total reward:",episode_reward)
    print("Frame Count = " + str(frame_count))
    
    if i%100 == 0:
        print("Saving the model")
        model.save("model/model-{}.h5".format(i))
        
        print("Saving the history")
        save_history("reward_history", history)

In [ ]:
# # หากต้องการ Run Colab โดยให้มีการรักษา Session ไว้ ท่านสามารถนำ Code ข้างล่างนี้ไปวางที่ JavaScript Console ของ Browser และสั่ง Run Script ได้ 
# function ConnectButton(){
#     console.log("Connect pushed"); 
#     document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
# }
# setInterval(ConnectButton,80000);